In [1]:
import os
import re
import pandas as pd

def extract_val(text, tag):
    """
    Simple tag-based pattern match for structured data like <us-gaap:Assets>.
    """
    pattern = fr"<us-gaap:{tag}.*?>(-?\(?\d[\d,]*\.?\d*\)?)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        val = match.group(1).replace(",", "")
        try:
            return float(val.replace("(", "-").replace(")", ""))
        except ValueError:
            return None
    return None

def extract_financials_to_excel(main_folder, output_excel):
    all_data = []

    for cik in os.listdir(main_folder):
        cik_path = os.path.join(main_folder, cik, "10-K")
        if not os.path.isdir(cik_path):
            continue

        for filing_folder in os.listdir(cik_path):
            file_path = os.path.join(cik_path, filing_folder, "full-submission.txt")
            if not os.path.exists(file_path):
                continue

            with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                data = file.read().rstrip().lower()

                try:
                    # Metadata
                    cik_val = cik
                    company_match = re.search(r'company conformed name:\s*\*?\s*(.+)', data)
                    company_name = company_match.group(1).strip() if company_match else "Unknown"

                    year_match = re.search(r"(\d{4})", filing_folder)
                    year = year_match.group(1) if year_match else "Unknown"

                    year_end_match = re.search(r'fiscal year end:\s*\*?\s*(\d{4})', data)
                    year_end = year_end_match.group(1) if year_end_match else "1231"
                    if len(year_end) == 4 and year != "Unknown":
                        year_end = f"{year}-{year_end[:2]}-{year_end[2:]}"  # Format: yyyy-mm-dd

                    # Financials
                    asset = extract_val(data, "Assets")
                    liability = extract_val(data, "Liabilities")
                    net_income = extract_val(data, "NetIncomeLoss") or extract_val(data, "ProfitLoss")

                    # Excess Tax Benefit
                    excess_tax = extract_val(data, "ExcessTaxBenefitFromStockBasedCompensation")
                    excess_tax_source = "Extracted"

                    if excess_tax is None:
                        deferred_tax = extract_val(data, "DeferredIncomeTaxExpenseBenefit")
                        current_tax = extract_val(data, "CurrentIncomeTaxExpense")
                        if deferred_tax is not None and current_tax is not None:
                            excess_tax = deferred_tax - current_tax
                            excess_tax_source = "Calculated"
                        else:
                            excess_tax_source = "Missing"
                    if excess_tax is not None:
                        excess_tax = round(excess_tax / 1_000_000, 2)

                    # Effective Tax Rate
                    eff_tax_rate = None
                    eff_tax_source = "Missing"
                    rate_match = re.search(r'effective tax rate[^a-z0-9]{0,30}(\d{1,3}\.\d{1,2})%', data)
                    if rate_match:
                        eff_tax_rate = float(rate_match.group(1))
                        eff_tax_source = "Extracted"
                    else:
                        tax_expense = extract_val(data, "IncomeTaxExpenseBenefit")
                        pretax_income = extract_val(data, "IncomeLossFromContinuingOperationsBeforeIncomeTaxes")
                        if tax_expense is not None and pretax_income:
                            try:
                                eff_tax_rate = round((tax_expense / pretax_income) * 100, 2)
                                eff_tax_source = "Calculated"
                            except ZeroDivisionError:
                                eff_tax_rate = None

                    all_data.append({
                        "CIK": cik_val,
                        "Company Name": company_name.upper(),
                        "Year": year,
                        "Year End": year_end,
                        "Assets": asset,
                        "Liabilities": liability,
                        "Net Income": net_income,
                        "Excess Tax Benefit (in millions)": excess_tax,
                        "Excess Tax Source": excess_tax_source,
                        "Effective Tax Rate (%)": eff_tax_rate,
                        "Effective Rate Source": eff_tax_source
                    })

                    print(f"✔ Extracted: {company_name} ({year})")

                except Exception as e:
                    print(f"⚠️ Error processing {file_path}: {e}")

    df = pd.DataFrame(all_data)
    df.to_excel(output_excel, index=False)
    print(f"\n✅ Extraction complete! Excel saved to:\n{output_excel}")


# Run it
extract_financials_to_excel(
    main_folder="C:\\Users\\mdfaz\\Downloads\\sec-edgar-filings",
    output_excel="C:\\Users\\mdfaz\\Downloads\\sec_summary_output.xlsx"
)

✔ Extracted: abbott laboratories (0001)
✔ Extracted: abbott laboratories (0001)
✔ Extracted: bk technologies corp (0001)
✔ Extracted: bk technologies corp (0001)
✔ Extracted: advanced micro devices inc (0000)
✔ Extracted: advanced micro devices inc (0000)
✔ Extracted: air products & chemicals, inc. (0000)
✔ Extracted: air products & chemicals, inc. (0000)
✔ Extracted: southern co (0000)
✔ Extracted: southern co (0000)
✔ Extracted: ceco environmental corp (0000)
✔ Extracted: ceco environmental corp (0000)
✔ Extracted: matson, inc. (0001)
✔ Extracted: matson, inc. (0001)
✔ Extracted: alico, inc. (0000)
✔ Extracted: alico, inc. (0000)
✔ Extracted: skyworks solutions, inc. (0000)
✔ Extracted: skyworks solutions, inc. (0000)
✔ Extracted: howmet aerospace inc. (0000)
✔ Extracted: howmet aerospace inc. (0000)
✔ Extracted: hess corp (0001)
✔ Extracted: hess corp (0001)
✔ Extracted: american airlines group inc. (0000)
✔ Extracted: american airlines group inc. (0000)
✔ Extracted: american electr

✔ Extracted: eqt corp (0000)
✔ Extracted: eqt corp (0000)
✔ Extracted: escalade inc (0001)
✔ Extracted: escalade inc (0001)
✔ Extracted: espey mfg & electronics corp (0001)
✔ Extracted: espey mfg & electronics corp (0001)
✔ Extracted: dmc global inc. (0000)
✔ Extracted: dmc global inc. (0000)
✔ Extracted: exxon mobil corp (0000)
✔ Extracted: exxon mobil corp (0000)
✔ Extracted: farmer brothers co (0000)
✔ Extracted: farmer brothers co (0000)
✔ Extracted: ferro corp (0000)
✔ Extracted: flexsteel industries inc (0000)
✔ Extracted: flexsteel industries inc (0000)
✔ Extracted: fmc corp (0000)
✔ Extracted: fmc corp (0000)
✔ Extracted: forward industries, inc. (0001)
✔ Extracted: forward industries, inc. (0001)
✔ Extracted: franklin electric co inc (0000)
✔ Extracted: franklin electric co inc (0000)
✔ Extracted: frequency electronics inc (0001)
✔ Extracted: frequency electronics inc (0001)
✔ Extracted: friedman industries inc (0001)
✔ Extracted: friedman industries inc (0001)
✔ Extracted: fu

✔ Extracted: national research corp (0001)
✔ Extracted: ncr corp (0000)
✔ Extracted: ncr corp (0000)
✔ Extracted: berkshire hathaway energy co (0001)
✔ Extracted: berkshire hathaway energy co (0001)
✔ Extracted: entergy corp /de/ (0000)
✔ Extracted: entergy corp /de/ (0000)
✔ Extracted: new york times co (0000)
✔ Extracted: new york times co (0000)
✔ Extracted: newpark resources inc (0000)
✔ Extracted: newpark resources inc (0000)
✔ Extracted: nordson corp (0000)
✔ Extracted: nordson corp (0000)
✔ Extracted: nordstrom inc (0000)
✔ Extracted: nordstrom inc (0000)
✔ Extracted: movado group inc (0000)
✔ Extracted: movado group inc (0000)
✔ Extracted: north european oil royalty trust (0000)
✔ Extracted: north european oil royalty trust (0000)
✔ Extracted: xcel energy inc (0000)
✔ Extracted: xcel energy inc (0000)
✔ Extracted: northern states power co /wi/ (0000)
✔ Extracted: northern states power co /wi/ (0000)
✔ Extracted: northwestern corp (0000)
✔ Extracted: northwestern corp (0000)
✔ E

✔ Extracted: universal electronics inc (0000)
✔ Extracted: universal electronics inc (0000)
✔ Extracted: universal corp /va/ (0000)
✔ Extracted: universal corp /va/ (0000)
✔ Extracted: vse corp (0000)
✔ Extracted: vse corp (0000)
✔ Extracted: veeco instruments inc (0001)
✔ Extracted: veeco instruments inc (0001)
✔ Extracted: v f corp (0000)
✔ Extracted: v f corp (0000)
✔ Extracted: village super market inc (0000)
✔ Extracted: village super market inc (0000)
✔ Extracted: dominion energy, inc (0000)
✔ Extracted: dominion energy, inc (0001)
✔ Extracted: vishay intertechnology inc (0000)
✔ Extracted: vishay intertechnology inc (0000)
✔ Extracted: volt information sciences, inc. (0000)
✔ Extracted: walmart inc. (0000)
✔ Extracted: walmart inc. (0000)
✔ Extracted: avista corp (0000)
✔ Extracted: avista corp (0000)
✔ Extracted: watsco inc (0001)
✔ Extracted: watsco inc (0001)
✔ Extracted: ww international, inc. (0000)
✔ Extracted: ww international, inc. (0001)
✔ Extracted: weis markets inc (0

✔ Extracted: lam research corp (0000)
✔ Extracted: lam research corp (0000)
✔ Extracted: noble romans inc (0001)
✔ Extracted: noble romans inc (0001)
✔ Extracted: neogen corp (0000)
✔ Extracted: neogen corp (0001)
✔ Extracted: cooper companies, inc. (0000)
✔ Extracted: cooper companies, inc. (0000)
✔ Extracted: electronic arts inc. (0000)
✔ Extracted: electronic arts inc. (0000)
✔ Extracted: anixa biosciences inc (0001)
✔ Extracted: anixa biosciences inc (0001)
✔ Extracted: dominion energy, inc (0000)
✔ Extracted: dominion energy, inc (0001)
✔ Extracted: yellow corp (0000)
✔ Extracted: yellow corp (0000)
✔ Extracted: cincinnati bell inc (0000)
✔ Extracted: cincinnati bell inc (0001)
✔ Extracted: graham corp (0000)
✔ Extracted: graham corp (0000)
✔ Extracted: regis corp (0000)
✔ Extracted: regis corp (0000)
✔ Extracted: hexcel corp /de/ (0000)
✔ Extracted: hexcel corp /de/ (0000)
✔ Extracted: value line inc (0001)
✔ Extracted: value line inc (0001)
✔ Extracted: unifirst corp (0000)
✔ Ex

...

✔ Extracted: curiositystream inc. (0001)
✔ Extracted: chargepoint holdings, inc. (0001)
✔ Extracted: chargepoint holdings, inc. (0001)
✔ Extracted: avepoint, inc. (0001)
✔ Extracted: avepoint, inc. (0001)
✔ Extracted: ironnet, inc. (0000)
✔ Extracted: ironnet, inc. (0000)
✔ Extracted: danimer scientific, inc. (0000)
✔ Extracted: danimer scientific, inc. (0000)
✔ Extracted: blade air mobility, inc. (0001)
✔ Extracted: wm technology, inc. (0001)
✔ Extracted: wm technology, inc. (0001)
✔ Extracted: lulu's fashion lounge holdings, inc. (0001)
✔ Extracted: lulu's fashion lounge holdings, inc. (0001)
✔ Extracted: ast spacemobile, inc. (0000)
✔ Extracted: ast spacemobile, inc. (0000)
✔ Extracted: acrivon therapeutics, inc. (0000)
✔ Extracted: blue water vaccines inc. (0001)
✔ Extracted: blue water vaccines inc. (0001)
✔ Extracted: remitly global, inc. (0001)
✔ Extracted: remitly global, inc. (0001)
✔ Extracted: pyxis oncology, inc. (0000)
✔ Extracted: pyxis oncology, inc. (0000)
✔ Extracted: 

✔ Extracted: american outdoor brands, inc. (0000)
✔ Extracted: alight, inc. / delaware (0000)
✔ Extracted: alight, inc. / delaware (0000)
✔ Extracted: goodrx holdings, inc. (0000)
✔ Extracted: goodrx holdings, inc. (0000)
✔ Extracted: edible garden ag inc (0001)
✔ Extracted: mirion technologies, inc. (0001)
✔ Extracted: mirion technologies, inc. (0001)
✔ Extracted: rackspace technology, inc. (0001)
✔ Extracted: rackspace technology, inc. (0001)
✔ Extracted: 1847 goedeker inc. (0001)
✔ Extracted: polished.com inc. (0001)
✔ Extracted: revelation biosciences, inc. (0000)
✔ Extracted: revelation biosciences, inc. (0001)
✔ Extracted: radius global infrastructure, inc. (0001)
✔ Extracted: radius global infrastructure, inc. (0001)
✔ Extracted: unity software inc. (0001)
✔ Extracted: unity software inc. (0001)
✔ Extracted: texas pacific land corp (0001)
✔ Extracted: texas pacific land corp (0001)
✔ Extracted: lucid group, inc. (0001)
✔ Extracted: lucid group, inc. (0001)
✔ Extracted: quantumsc

✔ Extracted: cf acquisition corp. iv (0001)
✔ Extracted: poshmark, inc. (0000)
✔ Extracted: latch, inc. (0001)
✔ Extracted: rover group, inc. (0001)
✔ Extracted: rover group, inc. (0001)
✔ Extracted: petco health & wellness company, inc. (0000)
✔ Extracted: petco health & wellness company, inc. (0001)
✔ Extracted: montauk renewables, inc. (0001)
✔ Extracted: montauk renewables, inc. (0001)
✔ Extracted: beachbody company, inc. (0000)
✔ Extracted: beachbody company, inc. (0000)
✔ Extracted: cvent holding corp. (0000)
✔ Extracted: cvent holding corp. (0000)
✔ Extracted: vigil neuroscience, inc. (0000)
✔ Extracted: vigil neuroscience, inc. (0000)
✔ Extracted: certara, inc. (0001)
✔ Extracted: certara, inc. (0001)
✔ Extracted: embark technology, inc. (0001)
✔ Extracted: embark technology, inc. (0001)
✔ Extracted: playtika holding corp. (0001)
✔ Extracted: playtika holding corp. (0001)
✔ Extracted: churchill capital corp vii (0001)
✔ Extracted: churchill capital corp vii (0001)
✔ Extracted: 

✔ Extracted: payoneer global inc. (0001)
✔ Extracted: payoneer global inc. (0001)
✔ Extracted: nextdoor holdings, inc. (0001)
✔ Extracted: nextdoor holdings, inc. (0001)
✔ Extracted: avista public acquisition corp. ii (0001)
✔ Extracted: omniab, inc. (0001)
✔ Extracted: membership collective group inc. (0000)
✔ Extracted: membership collective group inc. (0000)
✔ Extracted: 8i acquisition 2 corp. (0001)
✔ Extracted: euda health holdings ltd (0001)
✔ Extracted: centessa pharmaceuticals plc (0001)
✔ Extracted: centessa pharmaceuticals plc (0001)
✔ Extracted: verano holdings corp. (0001)
✔ Extracted: ryan specialty group holdings, inc. (0000)
✔ Extracted: ryan specialty holdings, inc. (0000)
✔ Extracted: cleantech acquisition corp. (0001)
✔ Extracted: nauticus robotics, inc. (0001)
✔ Extracted: century therapeutics, inc. (0001)
✔ Extracted: century therapeutics, inc. (0001)
✔ Extracted: akili, inc. (0000)
✔ Extracted: social capital suvretta holdings corp. i (0001)
✔ Extracted: prokidney 

✔ Extracted: d-wave quantum inc. (0001)
✔ Extracted: jushi holdings inc. (0001)
✔ Extracted: mobileye global inc. (0001)
✔ Extracted: asset entities inc. (0001)
✔ Extracted: asp isotopes inc. (0001)
✔ Extracted: granite ridge resources, inc. (0001)
✔ Extracted: rxo, inc. (0001)
✔ Extracted: ge healthcare technologies inc. (0001)
✔ Extracted: mineralys therapeutics, inc. (0001)
✔ Extracted: prosomnus, inc. (0001)
✔ Extracted: biohaven ltd. (0001)
✔ Extracted: zymeworks inc. (0001)
✔ Extracted: cadrenal therapeutics, inc. (0001)
✔ Extracted: masterbrand, inc. (0001)
✔ Extracted: crane co (0001)
✔ Extracted: ispire technology inc. (0001)
✔ Extracted: sitio royalties corp. (0000)
✔ Extracted: madison square garden entertainment corp. (0001)

✅ Extraction complete! Excel saved to:
C:\Users\mdfaz\Downloads\Prof Yurko\sec_summary_output3.xlsx
